In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Dropout, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.src.utils import pad_sequences
from sklearn.model_selection import train_test_split
from custom_preprocessor import preprocess

# Get data
Pandas is used to get the data from the project directory.
The data is a .csv file, that has been created by book texts and prepared to be listed in the file word-by-word.

In [23]:
df = pd.read_csv('./data/dataset_families.csv')
df = df.sample(frac=1.0, random_state=12)
df.head()

,Word,Language Family
45885,pressaient elles appelaient,Romance
146714,hyreni rozhazuje je,Slavic
141522,odehral tehda v,Slavic
65000,te equivocas severus,Romance
80966,die genossenschaft,Germanic


# Prepare data
Divide the data into X and y. X is input and y is the desired output.
One-Hot Encode it, so the language is categorical. This is done with Pandas.

In [27]:
# Getting X - the input values
X = df.iloc[:, :-1]
X.loc[:, 'Word'] = X['Word'].astype(str)
tokenizer = Tokenizer(char_level=True) # Used for turning the words into numbers
tokenizer.fit_on_texts(X['Word'].tolist()) # Giving Tokenizer the data it needs to learn how to encode the words
X_sequences = tokenizer.texts_to_sequences(X['Word'].tolist()) # Now the words are encoded into sequences
input_dim = max(len(seq) for seq in X_sequences)
print(input_dim)
X = pad_sequences(
    X_sequences, 
    maxlen=input_dim, 
    padding='post'
) # Making all input values be of same length
print(f"Rows: {len(X)}")

# Getting y - the output values
y_df = pd.get_dummies(df.iloc[:, -1:])
y = y_df.values
print(f"Amount of languages to identify: {len(y[1])}")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

40
Rows: 187795
Amount of languages to identify: 5


In [25]:
model = Sequential()
model.add(Conv1D(input_dim, 2, activation='relu'))  # Convolutional layer
tf.keras.layers.SimpleRNN(8, activation='relu')
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(len(y[1]), activation='softmax'))
adam = optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [26]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=2048,verbose=1)

Epoch 1/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 14s 176ms/step - accuracy: 0.2995 - loss: 1.6080 - val_accuracy: 0.4359 - val_loss: 1.3290
Epoch 2/50
31/74 ━━━━━━━━━━━━━━━━━━━━ 7s 165ms/step - accuracy: 0.4376 - loss: 1.3339

KeyboardInterrupt: 

# Loss evaluation

Seeing how the model's loss changes by the epoch

In [ ]:
%matplotlib inline
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss")
plt.legend(['Train', 'Val'], loc='upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')

# Loss evaluation

Seeing how the model's loss changes by the epoch

In [ ]:
%matplotlib inline
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Accuracy")
plt.legend(['Train', 'Val'], loc='upper right')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

# Prediction

In [28]:
prediction_word = ("Hvem er du og hvad laver du her")
print(f"Text before preprocessing: {prediction_word}")
prediction_word = preprocess(prediction_word)
print(f"Text after preprocessing: {prediction_word}")
prediction_word = tokenizer.texts_to_sequences([prediction_word])
prediction_word = pad_sequences(prediction_word, maxlen=input_dim)
prediction = model.predict([prediction_word])
output_probabilities = np.array(prediction)
result = {}
class_labels = np.unique(y_df.columns)
for index, prediction in enumerate(output_probabilities[0]):
    result.update({class_labels[index]: round(prediction * 100, 1)})

# Print keys and values of the result dictionary
for key, value in result.items():
    print(f"{key}: {value}%")

Text before preprocessing: Hvem er du og hvad laver du her
Text after preprocessing: hvem er du og hvad laver du her hvem er du og
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Language Family_Germanic: 78.7%
Language Family_Hellenic: 0.2%
Language Family_Romance: 5.0%
Language Family_Slavic: 1.9%
Language Family_Uralic: 14.3%


# Save model
Here a model can be loaded

In [ ]:
model.save('./models/model_lang_families.keras')

# Load model
Here a model can be loaded

In [ ]:
model = load_model('./models/model_lang_families.keras')